# 1. 環境設定

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Feb 28 17:47:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## google_drive

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')# google給的範例, 執行後會把google drive整個mount近來，路徑會多一個gdrive/MyDrive在content下
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
dataset  training_example


## darknet

In [3]:
# clone the Darknet
!git clone https://github.com/AlexeyAB/darknet
# 執行完後darknet資料夾會直接在content目錄下(因為現在就在content內)
%cd darknet
# 之後都在darknet資料夾裡
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make
# make darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14722, done.
remote: Total 14722 (delta 0), reused 0 (delta 0), pack-reused 14722
Receiving objects: 100% (14722/14722), 13.28 MiB | 24.76 MiB/s, done.
Resolving deltas: 100% (10015/10015), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_t

# 2. preprocessing

## data preprocessing

In [4]:
!mkdir dataset
!unzip /mydrive/dataset/images.zip -d dataset/

Archive:  /mydrive/dataset/images.zip
   creating: dataset/images/
  inflating: dataset/images/httpcsvkmeuaeccjpg.jpg  
  inflating: dataset/images/httpcsvkmeuaeccjpg.txt  
  inflating: dataset/images/httpftqncomymusicLxZeltonjohnjpg.jpg  
  inflating: dataset/images/httpftqncomymusicLxZeltonjohnjpg.txt  
  inflating: dataset/images/httpgraphicsnytimescomimagessectionmoviesfilmographyWireImagejpg.jpg  
  inflating: dataset/images/httpgraphicsnytimescomimagessectionmoviesfilmographyWireImagejpg.txt  
  inflating: dataset/images/httpiamediaimdbcomimagesMMVBMTAxNDUMzUwOTdeQTJeQWpwZBbWUMDUOTAyNTIVUXCRALjpg.jpg  
  inflating: dataset/images/httpiamediaimdbcomimagesMMVBMTAxNDUMzUwOTdeQTJeQWpwZBbWUMDUOTAyNTIVUXCRALjpg.txt  
  inflating: dataset/images/httpikinjaimgcomgawkermediaimageuploadsWmIuhdsrcedidjpgjpg.jpg  
  inflating: dataset/images/httpikinjaimgcomgawkermediaimageuploadsWmIuhdsrcedidjpgjpg.txt  
  inflating: dataset/images/httpimagescontactmusiccomnewsimagesjerryseinfeldjpg.jpg  
 

In [5]:
# 範例只train1個類別: 人臉
import glob
import os
import re
# process label
txt_file_paths = glob.glob(r"dataset/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)
# process image
images_list = glob.glob("dataset/images/*.jpg")
print(images_list)
train_list = images_list[:-2]
test_list = images_list[-2:]
#Create training.txt file
file = open("dataset/train.txt", "w") 
file.write("\n".join(train_list)) 
file.close() 

#Create test.txt file
file = open("dataset/test.txt", "w") 
file.write("\n".join(test_list)) 
file.close()
# debug
print("num of train images : {}".format(len(train_list)))
print("num of valid images : {}".format(len(test_list)))

0 0.684322 0.255892 0.258475 0.296296

['0', '0.684322', '0.255892', '0.258475', '0.296296']
0 dataset/images/httpssmediacacheakpinimgcomxdfdfadcfeabjpg.txt
0 0.684322 0.255892 0.258475 0.296296
1 0.415000 0.309406 0.230000 0.361386

['1', '0.415000', '0.309406', '0.230000', '0.361386']
1 dataset/images/httpftqncomymusicLxZeltonjohnjpg.txt
0 0.415000 0.309406 0.230000 0.361386
0 0.466463 0.370000 0.689024 0.668000

['0', '0.466463', '0.370000', '0.689024', '0.668000']
2 dataset/images/httpimagesfandangocomrImageRendererredesignstaticimgnoxportraitjpgpcpcpcimagesmasterrepositoryperformerimagespjpg.txt
0 0.466463 0.370000 0.689024 0.668000
1 0.467290 0.340694 0.626168 0.498423

['1', '0.467290', '0.340694', '0.626168', '0.498423']
3 dataset/images/httpiamediaimdbcomimagesMMVBMTAxNDUMzUwOTdeQTJeQWpwZBbWUMDUOTAyNTIVUXCRALjpg.txt
0 0.467290 0.340694 0.626168 0.498423
2 0.451667 0.232143 0.243333 0.278571

['2', '0.451667', '0.232143', '0.243333', '0.278571']
4 dataset/images/httpikinjaimgco

## training config

In [6]:
# Download pretrain weights darknet53
!wget https://pjreddie.com/media/files/darknet53.conv.74
!cp cfg/yolov3.cfg cfg/test_yolo.cfg

!sed -i 's/batch=1/batch=64/' cfg/test_yolo.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/test_yolo.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/test_yolo.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/test_yolo.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/test_yolo.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/test_yolo.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/test_yolo.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/test_yolo.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/test_yolo.cfg
# number of anchors = 3 in each output feature map
# number of classes = 1
# filters = (4+1+1)*3 = 18
!echo -e "face\n" > dataset/test_yolo.names
!echo -e 'classes= 1\ntrain  = dataset/train.txt\nvalid  = dataset/test.txt\nnames = dataset/test_yolo.names\nbackup = /mydrive/training_example/test_backup' > dataset/test_yolo.data

--2021-02-28 17:51:10--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  42.8MB/s    in 4.0s    

2021-02-28 17:51:15 (38.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# 3. training

In [7]:
# cp training config
!cp dataset/test_yolo.data /mydrive/training_example/test_yolo.data
!cp dataset/test_yolo.names /mydrive/training_example/test_yolo.names
!cp cfg/test_yolo.cfg /mydrive/training_example/test_yolo.cfg

In [ ]:
# Start the training
!./darknet detector train dataset/test_yolo.data cfg/test_yolo.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
test_yolo
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut 